### This notebook creates the embeddings using Sentence Transformers for Semantic IDs.

*In particular, it loads the .item file, creates a template to be fed to a sentence transformer and then save the embedding for each item in a .itememb file.*

This notebook currently support:
- RPG

**Load Libraries**

In [ ]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
import tiktoken
import torch
from tqdm import tqdm
from openai import OpenAI
import numpy as np
import re
import html
from typing import Union

**set the configuration**

In [ ]:
data_path = "/home/recsysdatasets"
dataset = "ml-1m_small"
sentence_trf_bs = 2048 # set the highest bs that fits in your GPU memory
sentence_trf_model = "sentence-transformers/sentence-t5-base" # size 768
apply_pca = True
sentence_trf_pca_components = 128
openai_api_key = None
separator = ' '

**specify which columns to use as metadata**


*they depend on the dataset. In this case, it's for ml-100k.*

In [ ]:
# la colonna feature non l'ho trovata nei metadata amazon.
columns_to_concatenate = ['movie_title', 'release_year', 'class']

**set the device used by the sentence transformer**

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

**load .item file used to retrieve item's metadata**

In [ ]:
item_feat_file = os.path.join(data_path, dataset, f'{dataset}.item')
item_feat_df = pd.read_csv(item_feat_file, sep='\t')

**Rename the columns**

In [ ]:
# Rename the columns by removing everything after ':'
new_columns = {}
for col in item_feat_df.columns:
    new_columns[col] = col.split(':')[0]
item_feat_df.rename(columns=new_columns, inplace=True)

**concatenate values and create the final dict**

In [ ]:
# utilities functions used to preprocess the feature data of an item.
def list_to_str(l: Union[list, str], remove_blank=False) -> str:
    """
    Converts a list or a string to a string representation.

    Args:
        l (Union[list, str]): The input list or string.

    Returns:
        str: The string representation of the input.
    """
    ret = ''
    if isinstance(l, list):
        ret = ', '.join(map(str, l))
    else:
        ret = l
    if remove_blank:
        ret = ret.replace(' ', '')
    return ret

def clean_text(raw_text: str) -> str:
    """
    Cleans the raw text by removing HTML tags, special characters, and extra spaces.

    Args:
        raw_text (str): The raw text to be cleaned.

    Returns:
        str: The cleaned text.
    """
    text = list_to_str(raw_text)
    text = html.unescape(text)
    text = text.strip()
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'[\n\t]', ' ', text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(r'[^\x00-\x7F]', ' ', text)
    return text

def _sent_process(raw: str) -> str:
    """
    Process the raw input according to the raw data type and return a processed sentence.

    Args:
        raw (str): The raw input to be processed.

    Returns:
        str: The processed sentence.
    """
    sentence = ""
    if isinstance(raw, float):
        sentence += str(raw)
        sentence += '.'
    elif len(raw) > 0 and isinstance(raw[0], list):
        for v1 in raw:
            for v in v1:
                sentence += clean_text(v)[:-1]
                sentence += ', '
        sentence = sentence[:-2]
        sentence += '.'
    elif isinstance(raw, list):
        for v1 in raw:
            sentence += clean_text(v1)
    else:
        sentence = clean_text(raw)
    return sentence + ' '

In [ ]:
if 'amazon' in dataset.lower():
    item_dict = dict()
    for index, row in item_feat_df.iterrows():
        item_id = row['item_id']
        meta_sentence = ''
        # Iterate through each column identified for concatenation
        for col in columns_to_concatenate:
            # Convert each value to string and handle potential NaN values by replacing them with an empty string
            meta_sentence += _sent_process(row[col])
        
        # Join the collected parts with a space and remove any leading/trailing whitespace
        item_dict[item_id] = meta_sentence
else:
    item_dict = dict()
    for index, row in item_feat_df.iterrows():
        item_id = row['item_id']
        concatenated_parts = []
        # Iterate through each column identified for concatenation
        for col in columns_to_concatenate:
            # Convert each value to string and handle potential NaN values by replacing them with an empty string
            value = row[col] if pd.notna(row[col]) else ''
            concatenated_parts.append(value)

        # convert everything to string
        concatenated_parts = [str(part) for part in concatenated_parts]
        # Join the collected parts with a space and remove any leading/trailing whitespace  
        item_dict[item_id] = separator.join(concatenated_parts).strip()

**item dict is a dictionary where the key is the item and the value is the description that will be used to obtain from the sentence trasformer the corresponding representation.**

In [ ]:
item_dict

**Load the Sentence Transformer**

In [ ]:
if 'sentence-transformer' in sentence_trf_model:
    print(f"Using {sentence_trf_model} for sentence embeddings")
    sentence_trf = SentenceTransformer(sentence_trf_model).to(device)

    sentence_embeddings = sentence_trf.encode(
        list(item_dict.values()),
        convert_to_numpy=True,
        batch_size=sentence_trf_bs,
        show_progress_bar=True,
        device=device
    )
else:
    print(f"Using OpenAI {sentence_trf_model} for sentence embeddings")
    client = OpenAI(api_key=openai_api_key)

    sentence_embeddings = []
    for i in tqdm(range(0, len(item_dict.values()), sentence_trf_bs), desc='Encoding: '):
        try:
            responses = client.embeddings.create(
                input=list(item_dict.values())[i: i + sentence_trf_bs],
                model=sentence_trf_model
            )
        except:
            print(f'Failed to encode sentence embeddings for {i} - {i + sentence_trf_bs}')
            batch = list(item_dict.values())[
                i: i + sentence_trf_bs]

            new_batch = []
            for sent in batch:
                
                encoding = tiktoken.get_encoding('cl100k_base')
                num_tokens = len(encoding.encode(sent))

                if num_tokens < 8192:
                    new_batch.append(sent)
                else:
                    n_chars = 8192 / num_tokens * len(sent) - 100
                    new_batch.append(sent[:int(n_chars)])

            print(f'Retrying with {len(new_batch)} sentences')
            responses = client.embeddings.create(
                input=new_batch,
                model=sentence_trf_model
            )

        for response in responses.data:
            sentence_embeddings.append(response.embedding)
    sentence_embeddings = np.array(sentence_embeddings, dtype=np.float32)

# for Amazon_Sports_and_Outdoors is about 31minutes

In [ ]:
if apply_pca:
    from sklearn.decomposition import PCA
    pca = PCA(n_components=sentence_trf_pca_components, whiten=True)
    sentence_embeddings = pca.fit_transform(sentence_embeddings)

**Save the representations**

In [ ]:
data = {
    'item_embedding_id:token': item_dict.keys(),
    'item_embedding:float_seq': sentence_embeddings.tolist(),
}
df = pd.DataFrame(data)
df['item_embedding:float_seq'] = df['item_embedding:float_seq'].apply(lambda x: ' '.join(map(str, x)))

In [ ]:
df

**Save the final file**

In [ ]:
df.to_csv(os.path.join(data_path,dataset,f'{dataset}.itememb'),sep="\t", index=False, header=True)

In [ ]:
os.listdir(os.path.join(data_path,dataset))